In [1]:
import pandas as pd 
import numpy as np 
import requests
import urllib.parse
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [2]:
# 파일 불러오기: 불러올 파일을 현재 폴더에 넣어두세요.
df = pd.read_csv('../programmers.csv')  # 파일명만 변경하여 그대로 돌리시면 됩니다.

In [3]:
# 웹 드라이버 초기화
driver = webdriver.Chrome()

# 기업명 리스트
company_list = df.기업명

# 데이터를 저장할 리스트 초기화
data = []

try:
    for company_name in company_list:
        formatted_keyword = urllib.parse.quote(company_name)  # 기업명을 URL 형식에 맞게 변환
        url = f"https://search.incruit.com/list/search.asp?col=company&kw={formatted_keyword}"
        
        # 검색 결과가 있는지 확인
        response = requests.get(url)
        if "검색결과가 없습니다" not in response.text:
            # URL로 이동
            driver.get(url)
            
            # 첫 번째 기업 링크를 찾기 위한 XPath
            first_company_xpath = './/div[1]/div/div[1]/span/a'
    
            # 첫 번째 기업 링크 클릭
            first_company_link = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, first_company_xpath))
            )
            first_company_link.click()
            
            # 타임슬립 추가: 페이지가 완전히 로드될 때까지 기다리기 위해 2초 대기
            time.sleep(2)
    
            # 새로 열린 페이지로 전환
            driver.switch_to.window(driver.window_handles[1])
            
            # 새로 열린 페이지에서 데이터 크롤링
            # 페이지 소스 가져오기
            page_source = driver.page_source
            
            # BeautifulSoup을 사용하여 파싱
            soup = BeautifulSoup(page_source, 'html.parser')
            
            try:
                # 기업명, 설립연도, 매출액, 사원수, 업종분류, 기업형태 추출
                table_li = soup.select_one('div.table_li')
                if table_li:
                    corp_name = company_name
                    corp_year = table_li.select_one('strong:contains("설립일") + span').text.strip()  # 설립일을 설립연도로 변경
                    sales_money = table_li.select_one('strong:contains("매출액") + span').text.strip()
                    employee_count = table_li.select_one('strong:contains("사원수") + span').text.strip()
                    sector = table_li.select_one('strong:contains("업종") + span').text.strip()  # 업종을 업종분류로 변경
                    # 기업형태는 없는 경우도 있을 수 있으므로 예외 처리
                    try:
                        company_scale = table_li.select_one('strong:contains("기업형태") + span').text.strip()
                    except AttributeError:
                        company_scale = None
                    
                    # 데이터 추가
                    data.append({
                        '기업명': corp_name,
                        '설립연도': corp_year,
                        '매출액': sales_money,
                        '사원수': employee_count,
                        '업종분류': sector,
                        '기업형태': company_scale
                    })
                else:
                    print("테이블 요소를 찾을 수 없습니다:", driver.current_url)
                
                # 주소와 홈페이지 URL 추출
                myCompany_info = soup.select_one('ul.myCompany_info')
                if myCompany_info:
                    address = myCompany_info.select_one('li:contains("주소")').text.strip().split(':')[-1].strip()
                    homepage = myCompany_info.select_one('li:contains("홈페이지")').text.strip().split(':')[-1].strip()
                    # 데이터 추가
                    data[-1]['주소'] = address
                    data[-1]['기업 홈페이지 URL'] = homepage  # 변수 이름을 기업 홈페이지 URL로 변경
                else:
                    print("회사 정보를 찾을 수 없습니다:", driver.current_url)
            except Exception as e:
                print(f"기업 정보를 크롤링하는 도중 오류가 발생했습니다: {e}")
                
            # 새로 열린 페이지 닫기
            driver.close()

            # 다시 처음 페이지로 전환
            driver.switch_to.window(driver.window_handles[0])

except Exception as e:
    print(f"검색 및 데이터 수집 도중 오류가 발생했습니다: {e}")

finally:
    # 웹 드라이버 종료
    driver.quit()

c:\ProgramData\anaconda3\Lib\site-packages\soupsieve\css_parser.py:872: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(


기업 정보를 크롤링하는 도중 오류가 발생했습니다: 'NoneType' object has no attribute 'text'
기업 정보를 크롤링하는 도중 오류가 발생했습니다: 'NoneType' object has no attribute 'text'
기업 정보를 크롤링하는 도중 오류가 발생했습니다: 'NoneType' object has no attribute 'text'
기업 정보를 크롤링하는 도중 오류가 발생했습니다: 'NoneType' object has no attribute 'text'
기업 정보를 크롤링하는 도중 오류가 발생했습니다: 'NoneType' object has no attribute 'text'
테이블 요소를 찾을 수 없습니다: https://www.incruit.com/company/362756
회사 정보를 찾을 수 없습니다: https://www.incruit.com/company/362756
기업 정보를 크롤링하는 도중 오류가 발생했습니다: 'NoneType' object has no attribute 'text'
기업 정보를 크롤링하는 도중 오류가 발생했습니다: 'NoneType' object has no attribute 'text'


In [6]:
# 데이터프레임 생성
df = pd.DataFrame(data)

In [8]:
# CSV 파일로 저장
df.to_csv('incruit_result.csv', index=False)